In [41]:
import numpy as np
import cv2

net = cv2.dnn.readNet("./v4/yolov4.weights", "./v4/yolov4.cfg")

classes = []
with open("./v4/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()
outputs_layers = [layer_names[i-1] for i in net.getUnconnectedOutLayers()]

len(classes)

80

In [42]:
#จำนวน class ที่มีใน coco.names
classes[:10]
classes

['person',
 'bicycle',
 'car',
 'motorbike',
 'aeroplane',
 'bus',
 'train',
 'truck',
 'boat',
 'traffic light',
 'fire hydrant',
 'stop sign',
 'parking meter',
 'bench',
 'bird',
 'cat',
 'dog',
 'horse',
 'sheep',
 'cow',
 'elephant',
 'bear',
 'zebra',
 'giraffe',
 'backpack',
 'umbrella',
 'handbag',
 'tie',
 'suitcase',
 'frisbee',
 'skis',
 'snowboard',
 'sports ball',
 'kite',
 'baseball bat',
 'baseball glove',
 'skateboard',
 'surfboard',
 'tennis racket',
 'bottle',
 'wine glass',
 'cup',
 'fork',
 'knife',
 'spoon',
 'bowl',
 'banana',
 'apple',
 'sandwich',
 'orange',
 'broccoli',
 'carrot',
 'hot dog',
 'pizza',
 'donut',
 'cake',
 'chair',
 'sofa',
 'pottedplant',
 'bed',
 'diningtable',
 'toilet',
 'tvmonitor',
 'laptop',
 'mouse',
 'remote',
 'keyboard',
 'cell phone',
 'microwave',
 'oven',
 'toaster',
 'sink',
 'refrigerator',
 'book',
 'clock',
 'vase',
 'scissors',
 'teddy bear',
 'hair drier',
 'toothbrush']

In [43]:
#สร้างค่่าสีของกรอบแบบสุ่ม
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [44]:
img = cv2.imread("/Users/kancode/Desktop/Vehicle_Registration_Detection/data_test/merlin_187087536_cdb4120e-a704-499c-86a2-7e68b4fdf6df-superJumbo.jpg")

img = cv2.resize(img, None, fx=0.8, fy=0.8)
height, width, channels = img.shape
img.shape

(1092, 1638, 3)

In [45]:
# Process image
img_size = (320, 320)
blob = cv2.dnn.blobFromImage(img, 1/255, img_size, (0, 0, 0), True, crop=False)
net.setInput(blob)
outputs = net.forward(outputs_layers)

In [46]:
# วนอ่าน output เรื่อยๆ

class_ids = []
confidences = []
boxes = []
for output in outputs:
    for detection in output:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

# คัดเลือกกรอปที่มีความน่าจะเป็นสูงสุด
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

# Frontend
font = cv2.FONT_HERSHEY_SIMPLEX
text_color = (255, 255, 255)

for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        conf = confidences[i]
        label = "{}: {:.1f}".format(label, conf*100)
        corlor = colors[i]

        t_size = cv2.getTextSize(label, font, 1, 1)[0]
        cv2.rectangle(img, (x, y), (x + w, y + h), corlor, 2)

        c2 = x + int(t_size[0]*0.8), y - int(t_size[1]*1.2)
        cv2.rectangle(img, (x, y), c2, corlor, -1)
        cv2.putText(img, label, (x, y-2), font, 0.7, text_color, 3)

cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()